## Authenticating on Google Earth Engine

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AX4XfWgXKXBR_DbeWWGtj_Kf8ZLkyTwhmkW-hT0zvApJQENNk3q9sVlTjWU



Successfully saved authorization token.


## Creating an image collection based on Gridded Population of the World Version 4.11

In [2]:
#https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_Population_Count#description
# 'l' is for visualization purposes
l = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count").first()

## Basic visualization of population count on map

In [3]:
# 'e' is to visualize population count
e = l.select('population_count')

In [4]:
from IPython.display import Image

#visualising basic population count on map

# Create a URL to the styled image for a region around France.
url = e.getThumbUrl({
  "max": 1000.0,
  "palette": [
    "ffffe7",
    "86a192",
    "509791",
    "307296",
    "2c4484",
    "000066"
  ],"dimensions":4096,
  "min": 0.0
})
print(url)

# Display the thumbnail land surface temperature in France.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/49068f3e80a22ac32782551349000840-d5b590fda47cf07c29de9168a17a0c94:getPixels

Please wait while the thumbnail loads, it may take a moment...


## Creating dataframe with population count according to coordinates

In [24]:
# 'd' is to create dataframe (the same as 'l' but excluding '.first()'
d= ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count")

In [173]:
scale = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count").first().projection().nominalScale()

In [189]:
import folium
lat, lon = 33.8938, 35.5018
#scale is in meters
scale = 1
u_poi = ee.Geometry.Point(lon, lat)

In [190]:
e_u_poi = d.select('population_count').getRegion(u_poi, scale).getInfo()

In [191]:
import pandas as pd
df = pd.DataFrame(e_u_poi)

In [192]:
df.columns = df.iloc[0]

In [193]:
df = df[1:]
df

,id,longitude,latitude,time,population_count
1,gpw_v4_population_count_rev11_2000_30_sec,35.501802,33.893799,946684800000,7817.134277
2,gpw_v4_population_count_rev11_2005_30_sec,35.501802,33.893799,1104537600000,9581.345703
3,gpw_v4_population_count_rev11_2010_30_sec,35.501802,33.893799,1262304000000,11743.712891
4,gpw_v4_population_count_rev11_2015_30_sec,35.501802,33.893799,1420070400000,14394.094727
5,gpw_v4_population_count_rev11_2020_30_sec,35.501802,33.893799,1577836800000,17642.628906


In [165]:
df.reset_index(inplace=True, drop=True)
df

,id,longitude,latitude,time,population_count
0,gpw_v4_population_count_rev11_2000_30_sec,35.501802,33.893799,946684800000,7817.134277
1,gpw_v4_population_count_rev11_2005_30_sec,35.501802,33.893799,1104537600000,9581.345703
2,gpw_v4_population_count_rev11_2010_30_sec,35.501802,33.893799,1262304000000,11743.712891
3,gpw_v4_population_count_rev11_2015_30_sec,35.501802,33.893799,1420070400000,14394.094727
4,gpw_v4_population_count_rev11_2020_30_sec,35.501802,33.893799,1577836800000,17642.628906


In [119]:
#selecting only 2020 population count
df = df.iloc[[4]]
df

,id,longitude,latitude,time,population_count
4,gpw_v4_population_count_rev11_2020_30_sec,35.510403,33.893436,1577836800000,26484.136719


## Miscellaneous (to visualize based on coordinates?) - NOT COMPLETE

In [ ]:
my_map = folium.Map(location=[lat, lon], scale=scale)
# Add the land cover to the map object.
my_map.add_ee_layer(e, vis_params, 'population_count')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [ ]:
#visualization parameters 

vis_params= {
  "max": 1000.0,
  "palette": [
    "ffffe7",
    "86a192",
    "509791",
    "307296",
    "2c4484",
    "000066"
  ],"dimensions":4096,
  "min": 0.0
}

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(e).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
lat, lon = 24.02273182331778, 25.499267231174024
scale = 7
u_poi = ee.Geometry.Point(lon, lat)
my_map = folium.Map(location=[lat, lon], scale=scale)

# Add the land cover to the map object.
my_map.add_ee_layer(e, vis_params, 'population_count')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

## Failed Wikipedia Location Based Population Scraping

In [194]:
import requests
from bs4 import BeautifulSoup

In [205]:
inputt='miami'
response = requests.get(
    url=f"https://en.wikipedia.org/wiki/{inputt}",
)

In [206]:
soup = BeautifulSoup(response.content, 'html.parser')

In [207]:
title = soup.findAll('tr', class_= 'mergedrow')
print(title)

[<tr class="mergedrow"><th class="infobox-label" scope="row"><a href="/wiki/U.S._state" title="U.S. state">State</a></th><td class="infobox-data"><a href="/wiki/Florida" title="Florida">Florida</a></td></tr>, <tr class="mergedrow"><th class="infobox-label" scope="row"><a href="/wiki/County_(United_States)" title="County (United States)">Constituent counties</a> (<a href="/wiki/List_of_counties_in_Florida" title="List of counties in Florida">County</a>)</th><td class="infobox-data"><a href="/wiki/Miami-Dade_County,_Florida" title="Miami-Dade County, Florida">Miami-Dade</a></td></tr>, <tr class="mergedrow"><th class="infobox-label" scope="row"><a href="/wiki/List_of_regions_of_the_United_States" title="List of regions of the United States">Region</a></th><td class="infobox-data"><a href="/wiki/South_Atlantic_states" title="South Atlantic states">South Atlantic</a></td></tr>, <tr class="mergedrow"><th class="infobox-label" scope="row"><a href="/wiki/Municipal_corporation" title="Municipal

In [208]:
a= []
for element in title:
    a.append(element.text)

In [209]:
type(title)

bs4.element.ResultSet

In [210]:
b=[]
for x in a: 
    if "Total" in x:
        b.append(x)

In [211]:
c = b[-1]

In [212]:
d=c.strip('\xa0•\xa0Total')

In [214]:
e=d.replace(' ', '')[:-3]
e

'442,241'